# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [71]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [87]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,37.49,88,64,2.71,GL,1722884042
1,1,blackmans bay,-43.0167,147.3167,50.74,74,100,1.99,AU,1722884043
2,2,kone,-21.0595,164.8658,58.03,78,8,3.67,NC,1722884044
3,3,puerto natales,-51.7236,-72.4875,34.25,86,40,2.30,CL,1722883853
4,4,makushino,55.2103,67.2442,64.22,84,100,7.38,RU,1722884047


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [90]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                      color='Humidity', alpha=0.6, 
                                      tiles='OSM', frame_width=800, frame_height=600,
                                      hover_cols=['City', 'Humidity'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [120]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 10 
ideal_temp_max = 90
ideal_humidity_max = 80 
ideal_cloudiness_max = 90

# Drop any rows with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) &
    (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Humidity"] <= ideal_humidity_max) &
    (city_data_df["Cloudiness"] <= ideal_cloudiness_max)
].dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,kone,-21.0595,164.8658,58.03,78,8,3.67,NC,1722884044
6,6,port elizabeth,-33.9180,25.5701,56.32,65,0,8.05,ZA,1722884049
10,10,puerto ayora,-0.7393,-90.3518,76.96,78,59,11.90,EC,1722884053
11,11,georgetown,5.4112,100.3354,84.13,78,20,2.30,MY,1722883736
13,13,belvedere marittimo,39.6179,15.8617,82.11,73,0,2.66,IT,1722884057
...,...,...,...,...,...,...,...,...,...,...
587,587,nova vicosa,-17.8919,-39.3719,76.82,64,0,8.46,BR,1722884688
588,588,armacao de buzios,-22.7469,-41.8817,84.36,78,0,24.94,BR,1722884690
590,590,mus,39.0000,41.7500,73.78,18,0,1.90,TR,1722884693
591,591,kanye,-24.9667,25.3327,52.63,22,5,6.64,BW,1722884694


### Step 3: Create a new DataFrame called `hotel_df`.

In [123]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,kone,NC,-21.0595,164.8658,78,
6,port elizabeth,ZA,-33.9180,25.5701,65,
10,puerto ayora,EC,-0.7393,-90.3518,78,
11,georgetown,MY,5.4112,100.3354,78,
13,belvedere marittimo,IT,39.6179,15.8617,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [126]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kone - nearest hotel: Pacifik Appartels
port elizabeth - nearest hotel: Waterford Hotel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
georgetown - nearest hotel: Page 63 hostel
belvedere marittimo - nearest hotel: Bouganville Palace
pisco - nearest hotel: La Portada
adamstown - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
ribeira grande - nearest hotel: Hospedaria JSF
tromso - nearest hotel: Clarion Collection Hotel Aurora
yellowknife - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
lihue - nearest hotel: Kauai Palms
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
vorgashor - nearest hotel: София
tukrah - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
nova sintra - nearest hotel: Residência Ka Dencho
port saint john's - nearest hotel: Outback Inn
port-aux-francais - nearest hotel: Keravel

,City,Country,Lat,Lng,Humidity,Hotel Name
2,kone,NC,-21.0595,164.8658,78,Pacifik Appartels
6,port elizabeth,ZA,-33.9180,25.5701,65,Waterford Hotel
10,puerto ayora,EC,-0.7393,-90.3518,78,Hostal La Mirada De Solitario George
11,georgetown,MY,5.4112,100.3354,78,Page 63 hostel
13,belvedere marittimo,IT,39.6179,15.8617,73,Bouganville Palace
...,...,...,...,...,...,...
587,nova vicosa,BR,-17.8919,-39.3719,64,Pousada Pontal da Barra
588,armacao de buzios,BR,-22.7469,-41.8817,78,Vila da Santa
590,mus,TR,39.0000,41.7500,18,No hotel found
591,kanye,BW,-24.9667,25.3327,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [132]:
%%capture --no-display

# Configure the map plot
hotel_df['hover_text'] = hotel_df.apply(lambda row: f"City: {row['City']}<br>Country: {row['Country']}<br>Hotel: {row['Hotel Name']}<br>Humidity: {row['Humidity']}%", axis=1)
map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', color='Humidity', size=100, hover_cols=['hover_text'])
# Display the map
hvplot.show(map_plot)